In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv('Chemical_Data-1.csv')

In [3]:
features = ['Rate/kg', 'Rarity', 'Date Purchased','Amount Purchased']
target = 'Amount Purchased'

In [4]:
df['Date Purchased'] = pd.to_datetime(df['Date Purchased'], format='%d-%m-%Y')

In [5]:
df['Days Since Purchase'] = (df['Date Purchased'] - df['Date Purchased'].min()).dt.days

In [6]:
X = df[features].values
y = df[target].values.reshape(-1, 1)

In [7]:
X[:, 2] = df['Days Since Purchase'].astype(float)

In [8]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

In [9]:
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [10]:
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        sequences.append(seq)
    return np.array(sequences)

In [11]:
sequence_length = 10
X_seq = create_sequences(X_scaled, sequence_length)
y_seq = create_sequences(y_scaled, sequence_length)[:, -1]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [13]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
25/25 [==============================] - 3s 10ms/step - loss: 0.1329
Epoch 2/20
25/25 [==============================] - 0s 9ms/step - loss: 0.0334
Epoch 3/20
25/25 [==============================] - 0s 9ms/step - loss: 0.0219
Epoch 4/20
25/25 [==============================] - 0s 13ms/step - loss: 0.0187
Epoch 5/20
25/25 [==============================] - 0s 10ms/step - loss: 0.0168
Epoch 6/20
25/25 [==============================] - 0s 10ms/step - loss: 0.0155
Epoch 7/20
25/25 [==============================] - 0s 11ms/step - loss: 0.0140
Epoch 8/20
25/25 [==============================] - 0s 12ms/step - loss: 0.0128
Epoch 9/20
25/25 [==============================] - 0s 10ms/step - loss: 0.0110
Epoch 10/20
25/25 [==============================] - 0s 10ms/step - loss: 0.0093
Epoch 11/20
25/25 [==============================] - 0s 10ms/step - loss: 0.0052
Epoch 12/20
25/25 [==============================] - 0s 8ms/step - loss: 0.0023
Epoch 13/20
25/25 [=====================

In [15]:
y_pred_scaled = model.predict(X_test)

7/7 [==============================] - 0s 3ms/step


In [16]:
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1))

In [17]:
mae = mean_absolute_error(y_actual, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 12.586407439877288


In [18]:
mse = mean_squared_error(y_actual, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 304.6733499903581


In [19]:
r2 = r2_score(y_actual, y_pred)
print(f'R2 score: {r2}')

R2 score: 0.9960896617876771
